#Import Library

In [6]:
!pip install optuna
!pip install catboost
!pip install scikeras

In [7]:
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np
from tqdm.notebook import tqdm
import re
from functools import partial
from scipy.stats import mode
import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns
import plotly.express as px
from sklearn.linear_model import Lasso, LinearRegression, Ridge, ElasticNet
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, FunctionTransformer, PowerTransformer, PolynomialFeatures, RobustScaler
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, SimpleImputer, IterativeImputer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, GridSearchCV, RepeatedStratifiedKFold, cross_val_score, cross_val_predict, RepeatedKFold
from sklearn.metrics import roc_auc_score, roc_curve, RocCurveDisplay, cohen_kappa_score, log_loss, f1_score, median_absolute_error, accuracy_score, mean_squared_error
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.feature_selection import RFE, RFECV
from sklearn.isotonic import IsotonicRegression
from sklearn.calibration import CalibrationDisplay
from sklearn.inspection import PartialDependenceDisplay, permutation_importance
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor, GradientBoostingRegressor, ExtraTreesRegressor, VotingRegressor, StackingRegressor, BaggingRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVR
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from tensorflow.keras.models import Sequential
from keras.callbacks import EarlyStopping, TerminateOnNaN
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam, SGD, Nadam, Adagrad, Adadelta, RMSprop
import tensorflow_probability as tfpw
from scikeras.wrappers import KerasRegressor
import optuna
import tensorflow as tf
from tensorflow import keras
import tensorflow_probability as tfp
import warnings
warnings.filterwarnings('ignore')
sns.set()
import matplotlib.pyplot as plt
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin,clone
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

# Preprocessing

In [8]:
label = pd.read_csv('label.csv')
id = label['id']
label['available_at'] = id
label = label.drop('id',axis=1).set_index('available_at')

label


,pm10 (μg/m³)
available_at,
2022-08-04 11:00:00_loc01,39.6
2022-08-04 12:00:00_loc01,40.5
2022-08-04 13:00:00_loc01,41.6
2022-08-04 14:00:00_loc01,44.9
2022-08-04 15:00:00_loc01,51.5
...,...
2023-09-30 19:00:00_loc10,69.8
2023-09-30 20:00:00_loc10,65.7
2023-09-30 21:00:00_loc10,60.1


In [9]:
airqual_loc01_train = pd.read_csv('airqual_loc01_train.csv')
airqual_loc01_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10141 entries, 0 to 10140
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   available_at              10141 non-null  object 
 1   carbon_monoxide (μg/m³)   9826 non-null   float64
 2   nitrogen_dioxide (μg/m³)  9737 non-null   float64
 3   sulphur_dioxide (μg/m³)   9770 non-null   float64
 4   ozone (μg/m³)             9841 non-null   float64
 5   dust (μg/m³)              9905 non-null   float64
 6   uv_index ()               9716 non-null   float64
 7   uv_index_clear_sky ()     9717 non-null   float64
dtypes: float64(7), object(1)
memory usage: 633.9+ KB


In [10]:
#concating airqual 1 sampai 10
def lag_and_concat_airqual_data(data):
    airqual_10 = pd.read_csv(f'airqual_loc10_{data}.csv')
    airqual_10['available_at'] = airqual_10['available_at'].map(lambda x: f'{x}_loc10')
    airqual_10 = airqual_10.set_index('available_at')
    for i in airqual_10.columns:
        airqual_10[i] = airqual_10[i].shift(periods=-4)


    for i in range(9,0,-1):
        airqual_loc = pd.read_csv(f'airqual_loc0{i}_{data}.csv')
        airqual_loc['available_at'] = airqual_loc['available_at'].map(lambda x: f'{x}_loc0{i}')
        airqual_loc = airqual_loc.set_index('available_at')
        for i in airqual_loc.columns:
            airqual_loc[i] = airqual_loc[i].shift(periods=-4)
        airqual_10 = pd.concat([airqual_loc,airqual_10], axis=0)

    return airqual_10

airqual_train = lag_and_concat_airqual_data('train')
airqual_test = lag_and_concat_airqual_data('test')


In [11]:
def lag_and_concat_weather_data(data):
    weather_10 = pd.read_csv(f'weather_loc10_{data}.csv')
    weather_10['available_at'] = weather_10['available_at'].map(lambda x: f'{x}_loc10')
    weather_10 = weather_10.set_index('available_at')
    for i in weather_10.columns:
        weather_10[i] = weather_10[i].shift(periods=-4)

    for i in range(9,0,-1):
        weather_loc = pd.read_csv(f'weather_loc0{i}_{data}.csv')
        weather_loc['available_at'] = weather_loc['available_at'].map(lambda x: f'{x}_loc0{i}')
        weather_loc = weather_loc.set_index('available_at')

        for i in weather_loc.columns:
            weather_loc[i] = weather_loc[i].shift(periods=-4)
        weather_10 = pd.concat([weather_loc,weather_10], axis=0)

    return weather_10

weather_train = lag_and_concat_weather_data('train')
weather_test = lag_and_concat_weather_data('test')

In [12]:
merged_train = pd.merge(airqual_train, weather_train, on='available_at', how='left')
merged_train

,carbon_monoxide (μg/m³),nitrogen_dioxide (μg/m³),sulphur_dioxide (μg/m³),ozone (μg/m³),dust (μg/m³),uv_index (),uv_index_clear_sky (),temperature_2m (°C),relative_humidity_2m (%),dew_point_2m (°C),apparent_temperature (°C),rain (mm),pressure_msl (hPa),surface_pressure (hPa),cloud_cover (%),cloud_cover_low (%),cloud_cover_mid (%),cloud_cover_high (%),wind_speed_10m (km/h),wind_speed_100m (km/h),wind_direction_10m (°),wind_direction_100m (°),wind_gusts_10m (km/h)
available_at,,,,,,,,,,,,,,,,,,,,,,,
2022-08-04 11:00:00_loc01,622.0,4.8,1.3,136.0,0.0,6.4,8.2,24.9,NaN,225.6,28.6,2.5,1012.9,942.4,88.0,51.0,20.0,100.0,6.1,7.2,3.0,360.0,22.0
2022-08-04 12:00:00_loc01,620.0,4.2,NaN,150.0,0.0,6.4,8.4,25.0,80.0,21.3,29.3,2.4,1012.1,941.6,81.0,23.0,52.0,98.0,NaN,10.7,32.0,33.0,24.8
2022-08-04 13:00:00_loc01,629.0,3.9,42.2,163.0,0.0,2.1,NaN,25.2,76.0,20.8,28.9,2.7,1011.1,940.8,100.0,48.0,66.0,100.0,7.6,9.4,3.0,360.0,23.8
2022-08-04 14:00:00_loc01,655.0,4.4,1.8,170.0,0.0,1.4,4.8,25.2,75.0,20.5,28.9,1.2,1010.5,940.2,100.0,2165.7,100.0,98.0,2.4,2.5,63.0,90.0,20.9
2022-08-04 15:00:00_loc01,691.0,5.4,2.2,174.0,0.0,0.6,2.3,22.3,90.0,20.7,26.3,7.1,1010.9,939.9,100.0,96.0,100.0,100.0,1.6,2.3,27.0,72.0,14.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-30 19:00:00_loc10,496.0,15.7,34.1,74.0,0.0,0.0,0.0,27.7,75.0,22.8,30.1,NaN,1011.3,1009.0,28.0,0.0,2.0,89.0,20.7,23.5,257.0,259.0,28.1
2023-09-30 20:00:00_loc10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-30 21:00:00_loc10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
merged_test = pd.merge(airqual_test, weather_test, on='available_at', how='left')
merged_test

,carbon_monoxide (μg/m³),nitrogen_dioxide (μg/m³),sulphur_dioxide (μg/m³),ozone (μg/m³),dust (μg/m³),uv_index (),uv_index_clear_sky (),temperature_2m (°C),relative_humidity_2m (%),dew_point_2m (°C),apparent_temperature (°C),rain (mm),pressure_msl (hPa),surface_pressure (hPa),cloud_cover (%),cloud_cover_low (%),cloud_cover_mid (%),cloud_cover_high (%),wind_speed_10m (km/h),wind_speed_100m (km/h),wind_direction_10m (°),wind_direction_100m (°),wind_gusts_10m (km/h)
available_at,,,,,,,,,,,,,,,,,,,,,,,
2023-10-01 00:00:00_loc01,928.0,12.3,24.3,92.0,0.0,0.0,0.0,21.7,89.0,19.8,24.9,NaN,1014.2,942.8,2600.6,8.0,22.0,98.0,3.6,2.9,217.0,263.0,6.5
2023-10-01 01:00:00_loc01,862.0,10.0,18.2,74.0,0.0,0.0,0.0,21.5,NaN,19.2,NaN,0.0,1013.5,942.1,NaN,0.0,3.0,99.0,3.4,2.8,198.0,220.0,7.2
2023-10-01 02:00:00_loc01,780.0,8.5,13.5,59.0,0.0,0.0,0.0,21.1,88.0,19.0,23.9,0.0,1013.2,941.8,32.0,3.0,0.0,99.0,4.0,4.4,185.0,171.0,8.6
2023-10-01 03:00:00_loc01,NaN,7.3,9.2,46.0,0.0,0.0,0.0,21.3,87.0,19.0,23.9,0.0,1012.9,941.5,47.0,19.0,0.0,100.0,5.0,7.6,180.0,180.0,11.9
2023-10-01 04:00:00_loc01,607.0,6.6,6.1,38.0,0.0,0.0,0.0,NaN,88.0,18.9,23.6,0.0,1012.9,NaN,35.0,6.0,0.0,99.0,5.1,8.0,188.0,188.0,12.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-15 19:00:00_loc10,423.0,16.4,16.4,30.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-15 20:00:00_loc10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-02-15 21:00:00_loc10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
columns = merged_train.columns
index = merged_train.index
imputer = IterativeImputer(random_state=42)

merged_train = imputer.fit_transform(merged_train)
merged_train = pd.DataFrame(merged_train, columns=columns,index=index)


merged_train = pd.merge(merged_train, label,on='available_at', how='left')
merged_train

,carbon_monoxide (μg/m³),nitrogen_dioxide (μg/m³),sulphur_dioxide (μg/m³),ozone (μg/m³),dust (μg/m³),uv_index (),uv_index_clear_sky (),temperature_2m (°C),relative_humidity_2m (%),dew_point_2m (°C),apparent_temperature (°C),rain (mm),pressure_msl (hPa),surface_pressure (hPa),cloud_cover (%),cloud_cover_low (%),cloud_cover_mid (%),cloud_cover_high (%),wind_speed_10m (km/h),wind_speed_100m (km/h),wind_direction_10m (°),wind_direction_100m (°),wind_gusts_10m (km/h),pm10 (μg/m³)
available_at,,,,,,,,,,,,,,,,,,,,,,,,
2022-08-04 11:00:00_loc01,622.00000,4.800000,1.300000,136.000000,0.000000,6.400000,8.200000,24.900000,241.885155,225.600000,28.600000,2.500000,1012.900000,942.400000,88.000000,51.000000,20.000000,100.000000,6.100000,7.200000,3.000000,360.000000,22.000000,39.6
2022-08-04 12:00:00_loc01,620.00000,4.200000,23.149100,150.000000,0.000000,6.400000,8.400000,25.000000,80.000000,21.300000,29.300000,2.400000,1012.100000,941.600000,81.000000,23.000000,52.000000,98.000000,22.584401,10.700000,32.000000,33.000000,24.800000,40.5
2022-08-04 13:00:00_loc01,629.00000,3.900000,42.200000,163.000000,0.000000,2.100000,6.850814,25.200000,76.000000,20.800000,28.900000,2.700000,1011.100000,940.800000,100.000000,48.000000,66.000000,100.000000,7.600000,9.400000,3.000000,360.000000,23.800000,41.6
2022-08-04 14:00:00_loc01,655.00000,4.400000,1.800000,170.000000,0.000000,1.400000,4.800000,25.200000,75.000000,20.500000,28.900000,1.200000,1010.500000,940.200000,100.000000,2165.700000,100.000000,98.000000,2.400000,2.500000,63.000000,90.000000,20.900000,44.9
2022-08-04 15:00:00_loc01,691.00000,5.400000,2.200000,174.000000,0.000000,0.600000,2.300000,22.300000,90.000000,20.700000,26.300000,7.100000,1010.900000,939.900000,100.000000,96.000000,100.000000,100.000000,1.600000,2.300000,27.000000,72.000000,14.400000,51.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-30 19:00:00_loc10,496.00000,15.700000,34.100000,74.000000,0.000000,0.000000,0.000000,27.700000,75.000000,22.800000,30.100000,0.745463,1011.300000,1009.000000,28.000000,0.000000,2.000000,89.000000,20.700000,23.500000,257.000000,259.000000,28.100000,69.8
2023-09-30 20:00:00_loc10,1606.51354,43.943769,23.483348,201.943423,0.203948,4.956403,7.099918,70.651579,241.865273,62.152974,75.787701,0.838907,3166.443604,2920.565861,153.098396,69.639887,68.916014,218.173795,22.682095,33.978975,528.864246,538.620226,65.227845,65.7
2023-09-30 21:00:00_loc10,1606.51354,43.943769,23.483348,201.943423,0.203948,4.956403,7.099918,70.651579,241.865273,62.152974,75.787701,0.838907,3166.443604,2920.565861,153.098396,69.639887,68.916014,218.173795,22.682095,33.978975,528.864246,538.620226,65.227845,60.1


In [15]:
columns = merged_test.columns
index = merged_test.index

merged_test = imputer.transform(merged_test)
merged_test = pd.DataFrame(merged_test, columns=columns,index=index)
merged_test

,carbon_monoxide (μg/m³),nitrogen_dioxide (μg/m³),sulphur_dioxide (μg/m³),ozone (μg/m³),dust (μg/m³),uv_index (),uv_index_clear_sky (),temperature_2m (°C),relative_humidity_2m (%),dew_point_2m (°C),apparent_temperature (°C),rain (mm),pressure_msl (hPa),surface_pressure (hPa),cloud_cover (%),cloud_cover_low (%),cloud_cover_mid (%),cloud_cover_high (%),wind_speed_10m (km/h),wind_speed_100m (km/h),wind_direction_10m (°),wind_direction_100m (°),wind_gusts_10m (km/h)
available_at,,,,,,,,,,,,,,,,,,,,,,,
2023-10-01 00:00:00_loc01,928.000000,12.300000,24.300000,92.000000,0.000000,0.000000,0.000000,21.700000,89.000000,19.800000,24.900000,0.981359,1014.200000,942.800000,2600.600000,8.000000,22.000000,98.000000,3.600000,2.900000,217.000000,263.000000,6.500000
2023-10-01 01:00:00_loc01,862.000000,10.000000,18.200000,74.000000,0.000000,0.000000,0.000000,21.500000,241.869899,19.200000,75.646952,0.000000,1013.500000,942.100000,149.134037,0.000000,3.000000,99.000000,3.400000,2.800000,198.000000,220.000000,7.200000
2023-10-01 02:00:00_loc01,780.000000,8.500000,13.500000,59.000000,0.000000,0.000000,0.000000,21.100000,88.000000,19.000000,23.900000,0.000000,1013.200000,941.800000,32.000000,3.000000,0.000000,99.000000,4.000000,4.400000,185.000000,171.000000,8.600000
2023-10-01 03:00:00_loc01,1611.676078,7.300000,9.200000,46.000000,0.000000,0.000000,0.000000,21.300000,87.000000,19.000000,23.900000,0.000000,1012.900000,941.500000,47.000000,19.000000,0.000000,100.000000,5.000000,7.600000,180.000000,180.000000,11.900000
2023-10-01 04:00:00_loc01,607.000000,6.600000,6.100000,38.000000,0.000000,0.000000,0.000000,70.310658,88.000000,18.900000,23.600000,0.000000,1012.900000,2929.617169,35.000000,6.000000,0.000000,99.000000,5.100000,8.000000,188.000000,188.000000,12.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-15 19:00:00_loc10,423.000000,16.400000,16.400000,30.000000,0.000000,0.000000,0.000000,70.332652,242.177183,62.060015,75.694820,0.815887,3171.705130,2918.774399,153.370645,70.079893,69.201711,218.360796,22.764453,34.010313,529.386859,538.952061,65.424482
2024-02-15 20:00:00_loc10,1606.513540,43.943769,23.483348,201.943423,0.203948,4.956403,7.099918,70.651579,241.865273,62.152974,75.787701,0.838907,3166.443604,2920.565861,153.098396,69.639887,68.916014,218.173795,22.682095,33.978975,528.864246,538.620226,65.227845
2024-02-15 21:00:00_loc10,1606.513540,43.943769,23.483348,201.943423,0.203948,4.956403,7.099918,70.651579,241.865273,62.152974,75.787701,0.838907,3166.443604,2920.565861,153.098396,69.639887,68.916014,218.173795,22.682095,33.978975,528.864246,538.620226,65.227845


# Feature Engineering

## Weather Feature

In [16]:
def wind_features(_df:pd.DataFrame) -> pd.DataFrame:
    df = _df.copy()

    wv10 = df.pop('wind_speed_10m (km/h)')
    wd_rad10 = df.pop('wind_direction_10m (°)')*np.pi / 180

    df['wind_x_10'] = wv10*np.cos(wd_rad10)
    df['wind_y_10'] = wv10*np.sin(wd_rad10)

    wv100 = df.pop('wind_speed_100m (km/h)')
    wd_rad100 = df.pop('wind_direction_100m (°)')*np.pi / 180

    df['wind_x_100'] = wv100*np.cos(wd_rad100)
    df['wind_y_100'] = wv100*np.sin(wd_rad100)

    return df

merged_train = wind_features(merged_train)
merged_test = wind_features(merged_test)

In [17]:
def pressure_diff(_df: pd.DataFrame) -> pd.DataFrame:
    df = _df.copy()
    df["pres_diff"] = df["pressure_msl (hPa)"]-df["surface_pressure (hPa)"]
    return df

merged_train = pressure_diff(merged_train)
merged_test = pressure_diff(merged_test)

In [18]:
def temperature_diff(_df: pd.DataFrame) -> pd.DataFrame:
  df = _df.copy()
  df["temp_diff"] = df['apparent_temperature (°C)'] - df['temperature_2m (°C)']
  return df

merged_train = temperature_diff(merged_train)
merged_test = temperature_diff(merged_test)

## Airqual Feature

In [19]:
def polynomial_subs(_df: pd.DataFrame) -> pd.DataFrame :
    df = _df.copy()
    subs_lst = ["carbon_monoxide (μg/m³)", "nitrogen_dioxide (μg/m³)", "sulphur_dioxide (μg/m³)", "ozone (μg/m³)"]

    for subs in subs_lst :
        df[f"{subs}_squared"] = df[subs]**2

    return df

merged_train = polynomial_subs(merged_train)
merged_test = polynomial_subs(merged_test)

In [20]:
def interaction_terms(_df: pd.DataFrame) -> pd.DataFrame :
  df = _df.copy()

  df["no2o3_interaction"] =  df["nitrogen_dioxide (μg/m³)"] * df["ozone (μg/m³)"]
  df["so2no_interaction"] = df["nitrogen_dioxide (μg/m³)"] * df["sulphur_dioxide (μg/m³)"]
  df["no2co_intercation"] = df["nitrogen_dioxide (μg/m³)"] * df["carbon_monoxide (μg/m³)"]
  df["no2o3_interaction"] =  df["carbon_monoxide (μg/m³)"] * df["ozone (μg/m³)"]

  return df

merged_train = interaction_terms(merged_train)
merged_test = interaction_terms(merged_test)

In [21]:
from typing import List
lst = ['ozone (μg/m³)','carbon_monoxide (μg/m³)','nitrogen_dioxide (μg/m³)','sulphur_dioxide (μg/m³)']

def temporal_agg(_df: pd.DataFrame, lst: List[str], window_size: int = 3) -> pd.DataFrame:
    df = _df.copy()

    for col in lst:
        df[f'{col}_rolling_min'] = df[col].rolling(window=window_size, min_periods=1).min()
        df[f'{col}_rolling_mean'] = df[col].rolling(window=window_size, min_periods=1).mean()
        df[f'{col}_rolling_max'] = df[col].rolling(window=window_size, min_periods=1).max()

    return df

merged_train = temporal_agg(merged_train, lst)
merged_test = temporal_agg(merged_test, lst)



In [22]:
def uv_feature(_df: pd.DataFrame) -> pd.DataFrame :
  df = _df.copy()

  df['UV_impact'] = df['uv_index ()'] * df['ozone (μg/m³)']
  df['clear_sky_UV_impact'] = df['uv_index_clear_sky ()'] * df['ozone (μg/m³)']

  return df

merged_train = uv_feature(merged_train)
merged_test = uv_feature(merged_test)

In [23]:
def dayoftheweek(_df:pd.DataFrame) -> pd.DataFrame:
  df = _df.copy()
  index = df.index
  df.index = pd.to_datetime(df.index.str.split('_').str[0], format='%Y-%m-%d %H:%M:%S')
  df['weekday'] = df.index.weekday
  df['is_weekend'] = (df['weekday'] >= 5)
  df = df.set_index(index)
  return df

merged_train = dayoftheweek(merged_train)
merged_test = dayoftheweek(merged_test)

In [24]:
def cyclical_weekday(_df: pd.DataFrame) -> pd.DataFrame:
    df = _df.copy()
    df['day_of_week_sin'] = np.sin(2 * np.pi * df['weekday'] / 7)
    df['day_of_week_cos'] = np.cos(2 * np.pi * df['weekday'] / 7)
    return df

merged_train = cyclical_weekday(merged_train)
merged_test = cyclical_weekday(merged_test)

In [25]:
def nitrogen_sulphur_ratio(_df: pd.DataFrame) -> pd.DataFrame :
    df = _df.copy()
    df['nit/sulph_ratio'] = np.where((df['nitrogen_dioxide (μg/m³)'] != 0) & (df['sulphur_dioxide (μg/m³)'] != 0), df['nitrogen_dioxide (μg/m³)']/df['sulphur_dioxide (μg/m³)'], 0)

    return df

merged_train = nitrogen_sulphur_ratio(merged_train)
merged_test = nitrogen_sulphur_ratio(merged_test)

In [26]:
def carbon_ozone_ratio(_df: pd.DataFrame) -> pd.DataFrame :
    df = _df.copy()
    df['carb/ozone_ratio'] = np.where((df['carbon_monoxide (μg/m³)'] != 0) & (df['ozone (μg/m³)'] != 0), df['carbon_monoxide (μg/m³)']/df['ozone (μg/m³)'], 0)

    return df

merged_train = carbon_ozone_ratio(merged_train)
merged_test = carbon_ozone_ratio(merged_test)

In [27]:
def location(_df: pd.DataFrame) -> pd.DataFrame:
    df = _df.copy()
    df['location'] = df.index.str[-2:].map(lambda x: int(x))
    return df

merged_train = location(merged_train)
merged_test = location(merged_test)

In [28]:
merged_train.head()

,carbon_monoxide (μg/m³),nitrogen_dioxide (μg/m³),sulphur_dioxide (μg/m³),ozone (μg/m³),dust (μg/m³),uv_index (),uv_index_clear_sky (),temperature_2m (°C),relative_humidity_2m (%),dew_point_2m (°C),apparent_temperature (°C),rain (mm),pressure_msl (hPa),surface_pressure (hPa),cloud_cover (%),cloud_cover_low (%),cloud_cover_mid (%),cloud_cover_high (%),wind_gusts_10m (km/h),pm10 (μg/m³),wind_x_10,wind_y_10,wind_x_100,wind_y_100,pres_diff,temp_diff,carbon_monoxide (μg/m³)_squared,nitrogen_dioxide (μg/m³)_squared,sulphur_dioxide (μg/m³)_squared,ozone (μg/m³)_squared,no2o3_interaction,so2no_interaction,no2co_intercation,ozone (μg/m³)_rolling_min,ozone (μg/m³)_rolling_mean,ozone (μg/m³)_rolling_max,carbon_monoxide (μg/m³)_rolling_min,carbon_monoxide (μg/m³)_rolling_mean,carbon_monoxide (μg/m³)_rolling_max,nitrogen_dioxide (μg/m³)_rolling_min,nitrogen_dioxide (μg/m³)_rolling_mean,nitrogen_dioxide (μg/m³)_rolling_max,sulphur_dioxide (μg/m³)_rolling_min,sulphur_dioxide (μg/m³)_rolling_mean,sulphur_dioxide (μg/m³)_rolling_max,UV_impact,clear_sky_UV_impact,weekday,is_weekend,day_of_week_sin,day_of_week_cos,nit/sulph_ratio,carb/ozone_ratio,location
available_at,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2022-08-04 11:00:00_loc01,622.0,4.8,1.3000,136.0,0.0,6.4,8.200000,24.9,241.885155,225.6,28.6,2.5,1012.9,942.4,88.0,51.0,20.0,100.0,22.0,39.6,6.091640,0.319249,7.200000e+00,-1.763491e-15,70.5,3.7,386884.0,23.04,1.690000,18496.0,84592.0,6.24000,2985.6,136.0,136.000000,136.0,622.0,622.000000,622.0,4.8,4.800000,4.8,1.3,1.300000,1.3000,870.4,1115.200000,3,False,0.433884,-0.900969,3.692308,4.573529,1
2022-08-04 12:00:00_loc01,620.0,4.2,23.1491,150.0,0.0,6.4,8.400000,25.0,80.000000,21.3,29.3,2.4,1012.1,941.6,81.0,23.0,52.0,98.0,24.8,40.5,19.152658,11.967909,8.973775e+00,5.827638e+00,70.5,4.3,384400.0,17.64,535.880835,22500.0,93000.0,97.22622,2604.0,136.0,143.000000,150.0,620.0,621.000000,622.0,4.2,4.500000,4.8,1.3,12.224550,23.1491,960.0,1260.000000,3,False,0.433884,-0.900969,0.181433,4.133333,1
2022-08-04 13:00:00_loc01,629.0,3.9,42.2000,163.0,0.0,2.1,6.850814,25.2,76.000000,20.8,28.9,2.7,1011.1,940.8,100.0,48.0,66.0,100.0,23.8,41.6,7.589584,0.397753,9.400000e+00,-2.302336e-15,70.3,3.7,395641.0,15.21,1780.840000,26569.0,102527.0,164.58000,2453.1,136.0,149.666667,163.0,620.0,623.666667,629.0,3.9,4.300000,4.8,1.3,22.216367,42.2000,342.3,1116.682635,3,False,0.433884,-0.900969,0.092417,3.858896,1
2022-08-04 14:00:00_loc01,655.0,4.4,1.8000,170.0,0.0,1.4,4.800000,25.2,75.000000,20.5,28.9,1.2,1010.5,940.2,100.0,2165.7,100.0,98.0,20.9,44.9,1.089577,2.138416,1.530808e-16,2.500000e+00,70.3,3.7,429025.0,19.36,3.240000,28900.0,111350.0,7.92000,2882.0,150.0,161.000000,170.0,620.0,634.666667,655.0,3.9,4.166667,4.4,1.8,22.383033,42.2000,238.0,816.000000,3,False,0.433884,-0.900969,2.444444,3.852941,1
2022-08-04 15:00:00_loc01,691.0,5.4,2.2000,174.0,0.0,0.6,2.300000,22.3,90.000000,20.7,26.3,7.1,1010.9,939.9,100.0,96.0,100.0,100.0,14.4,51.5,1.425610,0.726385,7.107391e-01,2.187430e+00,71.0,4.0,477481.0,29.16,4.840000,30276.0,120234.0,11.88000,3731.4,163.0,169.000000,174.0,629.0,658.333333,691.0,3.9,4.566667,5.4,1.8,15.400000,42.2000,104.4,400.200000,3,False,0.433884,-0.900969,2.454545,3.971264,1


# Modelling

In [29]:
def smape(y_true, y_pred):
    assert len(y_true) == len(y_pred), "Input arrays must have the same length."

    numerator = 2 * np.abs(np.array(y_pred) - np.array(y_true))
    denominator = np.abs(np.array(y_pred)) + np.abs(np.array(y_true))

    denominator = np.where(denominator == 0, 1e-9, denominator)

    smape_value = np.mean(numerator / denominator) * 100

    return smape_value


In [30]:
X, y = merged_train.drop(['pm10 (μg/m³)'],axis=1), merged_train['pm10 (μg/m³)']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

## First Model

In [31]:
cb = CatBoostRegressor(
    **{'iterations': 997,
       'depth': 9,
       'learning_rate': 0.08619161640851952,
       'l2_leaf_reg': 9.880115363592025e-05,
       'min_child_samples': 19,
       'border_count': 70,
       }
)

lgb = LGBMRegressor(**{
        "verbose": -1,
        "num_leaves": 274,
        "learning_rate": 0.0363659868794739,
        "feature_fraction": 0.6288061125424013,
        "min_child_samples": 9,
        "max_depth": 20,
        "lambda_l1": 0.00016628985509900782,
        "lambda_l2": 0.002837398285557587,
        "min_split_gain": 0.7577793742601943,
        "min_child_weight": 9.419284482047434,
        "colsample_bytree": 0.7269574870557697,
        "subsample": 0.272828878529134,
        "reg_alpha": 6.174076387406154,
        "reg_lambda": 6.096821117413743,
        "n_estimators": 217
      }
)
xgb = XGBRegressor(
    **{
        "colsample_bytree": 0.5920623339546657,
        "learning_rate": 0.05655179773541492,
        "max_depth": 8,
        "n_estimators": 477,
        "gamma": 0.6651835678620678,
        "min_child_weight": 1.561555677120107,
        "alpha": 0.131264100671468,
        "lambda": 0.4116178861256159,
    }
)
rnf = RandomForestRegressor()

In [32]:
votingreg = VotingRegressor(estimators=[('cb',cb), ('lgb',lgb),('xgb',xgb),('rnf',rnf)])

In [33]:
smape_scorer_2 = make_scorer(smape, greater_is_better=False)

cross_val_scores2 = cross_val_score(votingreg, X, y, cv=5, scoring=smape_scorer_2, error_score='raise')

print(f"Mean SMAPE: {np.mean(-cross_val_scores2):.2f}")


0:	learn: 23.3196671	total: 190ms	remaining: 3m 9s
1:	learn: 21.7884723	total: 303ms	remaining: 2m 30s
2:	learn: 20.4292465	total: 445ms	remaining: 2m 27s
3:	learn: 19.1873468	total: 589ms	remaining: 2m 26s
4:	learn: 18.0637947	total: 708ms	remaining: 2m 20s
5:	learn: 17.0650973	total: 799ms	remaining: 2m 11s
6:	learn: 16.1746204	total: 894ms	remaining: 2m 6s
7:	learn: 15.3763934	total: 976ms	remaining: 2m
8:	learn: 14.6560553	total: 1.05s	remaining: 1m 55s
9:	learn: 14.0083859	total: 1.16s	remaining: 1m 54s
10:	learn: 13.4313151	total: 1.28s	remaining: 1m 55s
11:	learn: 12.9175265	total: 1.41s	remaining: 1m 55s
12:	learn: 12.4656937	total: 1.53s	remaining: 1m 55s
13:	learn: 12.0558822	total: 1.64s	remaining: 1m 55s
14:	learn: 11.6844552	total: 1.75s	remaining: 1m 54s
15:	learn: 11.3693630	total: 1.86s	remaining: 1m 54s
16:	learn: 11.0775908	total: 1.94s	remaining: 1m 51s
17:	learn: 10.8235109	total: 2.02s	remaining: 1m 49s
18:	learn: 10.6049061	total: 2.14s	remaining: 1m 50s
19:	learn

In [ ]:
print(-cross_val_scores2)

print(f"SMAPE: {np.std(-cross_val_scores2):.2f}")

[33.27470481 18.48311396 20.07847717 15.45229261 17.48897651]
SMAPE: 6.34


##Second Model

In [ ]:
lgb = LGBMRegressor(**{
        "verbose": -1,
        "num_leaves": 274,
        "learning_rate": 0.0363659868794739,
        "feature_fraction": 0.6288061125424013,
        "min_child_samples": 9,
        "max_depth": 20,
        "lambda_l1": 0.00016628985509900782,
        "lambda_l2": 0.002837398285557587,
        "min_split_gain": 0.7577793742601943,
        "min_child_weight": 9.419284482047434,
        "colsample_bytree": 0.7269574870557697,
        "subsample": 0.272828878529134,
        "reg_alpha": 6.174076387406154,
        "reg_lambda": 6.096821117413743,
        "n_estimators": 217
      }
)

lgb.fit(X_train,y_train)
y_pred = lgb.predict(X_test)

  File "c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "c:\Users\Asus\AppData\Local\Programs\Python\Python311\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


In [ ]:
score = smape(y_test,y_pred)
score

15.651961888565557

In [ ]:
smape_scorer = make_scorer(smape, greater_is_better=False)

cross_val_scores = cross_val_score(lgb, X, y, cv=5, scoring=smape_scorer)

print(f"Mean SMAPE: {np.mean(-cross_val_scores):.2f}")

Mean SMAPE: 21.23


In [ ]:
# Print the mean SMAPE score
print(-cross_val_scores)

print(f"SMAPE: {np.std(-cross_val_scores):.2f}")

[33.06661864 18.7471405  20.1307168  16.40334171 17.78209743]
SMAPE: 6.04


# Submission

In [34]:
votingreg.fit(X,y)
y_pred = votingreg.predict(merged_test)

0:	learn: 27.5061199	total: 67.2ms	remaining: 1m 6s
1:	learn: 25.8462205	total: 126ms	remaining: 1m 2s
2:	learn: 24.3230064	total: 179ms	remaining: 59.4s
3:	learn: 22.9810197	total: 238ms	remaining: 59.1s
4:	learn: 21.7699386	total: 298ms	remaining: 59.2s
5:	learn: 20.6607872	total: 360ms	remaining: 59.5s
6:	learn: 19.6465733	total: 417ms	remaining: 59s
7:	learn: 18.7305679	total: 484ms	remaining: 59.8s
8:	learn: 17.8955310	total: 540ms	remaining: 59.3s
9:	learn: 17.1221908	total: 598ms	remaining: 59s
10:	learn: 16.4333025	total: 655ms	remaining: 58.7s
11:	learn: 15.8221768	total: 710ms	remaining: 58.3s
12:	learn: 15.2816812	total: 766ms	remaining: 58s
13:	learn: 14.8243962	total: 826ms	remaining: 58s
14:	learn: 14.4145140	total: 883ms	remaining: 57.8s
15:	learn: 14.0334485	total: 939ms	remaining: 57.6s
16:	learn: 13.6828873	total: 992ms	remaining: 57.2s
17:	learn: 13.3914987	total: 1.05s	remaining: 57s
18:	learn: 13.1202456	total: 1.1s	remaining: 56.6s
19:	learn: 12.8752654	total: 1.1

In [35]:
sample = pd.read_csv('sample_submission.csv')
sample['pm10 (μg/m³)'] = y_pred
sample = sample.set_index('id')
sample

,pm10 (μg/m³)
id,
2023-10-01 00:00:00_loc01,119.472179
2023-10-01 01:00:00_loc01,93.112804
2023-10-01 02:00:00_loc01,84.318215
2023-10-01 03:00:00_loc01,77.094144
2023-10-01 04:00:00_loc01,51.860745
...,...
2024-02-15 19:00:00_loc10,25.442461
2024-02-15 20:00:00_loc10,46.520471
2024-02-15 21:00:00_loc10,49.700274


In [ ]:
sample.to_csv('submission.csv')

In [38]:
#save the model
import pickle

with open('votingreg.pkl', 'wb') as file:
    pickle.dump(votingreg, file)

In [39]:
with open('model_cb.pkl', 'wb') as file:
    pickle.dump(cb, file)

with open('model_lgb.pkl', 'wb') as file:
    pickle.dump(lgb, file)

with open('model_xgb.pkl', 'wb') as file:
    pickle.dump(xgb, file)

with open('model_rnf.pkl', 'wb') as file:
    pickle.dump(rnf, file)

In [40]:
with open('model_cb.pkl', 'rb') as file:
    cb_loaded = pickle.load(file)

with open('model_lgb.pkl', 'rb') as file:
    lgb_loaded = pickle.load(file)

with open('model_xgb.pkl', 'wb') as file:
    pickle.dump(xgb, file)

with open('model_rnf.pkl', 'wb') as file:
    pickle.dump(rnf, file)